# Reference
+ https://www.cnblogs.com/Imageshop/p/3281703.html

# Question
## 什么是暗通道？
在绝大多数非天空的局部区域里，某些像素总会有某个颜色通道很低的值，用数学表达式定义的话就是$J^{dark}(x)=\min_{y\in \Omega (x)}(\min_{c\in \{r,g,b\}}J^c(y))$
，其中$J^c$表示彩色图像的每一个通道，$\Omega (x)$表示以像素x为中心的一个窗口。实际计算过程中可以理解为，首先对图像求出每一个像素RGB三通道中的最小值，然后进行以此最小值滤波,即可求出$J^{dark}$
## 暗通道有什么先验知识？
$J^{dark}-->0$
## 如何利用暗通道进行去雾？
首先了解雾图像形成模型：$I(x)=J(x)t(x)+A(1-t(x))$，其中$I(x)$为雾图,$J(x)$为无雾图，A是全球大气光成分，$t(x)$为透射率。

接下来就是推导过程：
<img src=./img/darkChannel.jpeg>
上述推论中都是假设全球达气光A值时已知的，在实际中，我们可以借助于暗通道图来从有雾图像中获取该值。具体步骤如下：

1、从**暗通道**图中按照亮度的大小取前0.1%的像素。

2、在这些位置中，在**原始有雾图像I**中寻找对应的具有最高亮度的点的值，作为A值。

到这一步，我们就可以进行无雾图像的恢复了。由上式可知：J=(I-A)/t+A  
现在I,A,t都已经求得了，因此，完全可以进行J的计算。

当投射图t的值很小时，会导致J的值偏大，从而使淂图像整体向白场过度，因此一般可设置一阈值T0，当t值小于T0时，令t=T0，本文中所有效果图均以T0=0.1为标准计算。
因此最终的恢复公式：
$J(x)=\frac{I(x)-A}{max(t(x),t_0)}+A$


In [ ]:
import numpy as np
import cv2
from itertools import combinations_with_replacement
from collections import defaultdict
from numpy.linalg import inv

'''
J : haze free image
I : hazy image
t : 0 for haze free image and 1 for maximum hazy image
'''

class Dehazer:
	def __init__(self,img,omega=0.95,t0=0.1,w=5,w_refine=40,epsilon=1e-2,refine=True,threshold_percentage=0.001,show_intermediate=False):
		self.image = img # Corresponds to I in the paper
		self.omega = omega
		self.t0 = t0
		self.w = w
		self.w_refine = w_refine
		self.epsilon = epsilon
		self.refine = refine
		self.show_intermediate = show_intermediate
		self.threshold_percentage = threshold_percentage

	def get_dark_channel(self):
		'''
		w : window size
		Add docstring
		'''
		w = self.w
		self.dark_channel = np.zeros((self.image.shape[0],self.image.shape[1],1))
		padded_image = np.pad(self.image,((int(w/2),int(w/2)),(int(w/2),int(w/2)),(0,0)),'edge')

		for i in range(int(w/2),padded_image.shape[0] - int(w/2)):
			for j in range(int(w/2),padded_image.shape[1] - int(w/2)):
				block = padded_image[i-int(w/2):i+int(w/2)+1,j-int(w/2):j+int(w/2)+1,:]
				self.dark_channel[i-int(w/2),j-int(w/2),0] = np.min(block)

		# if self.show_intermediate is True:
		# 	cv2.imshow('Dark Channel Image',self.dark_channel)
		# 	cv2.waitKey(0)

	def get_A(self):
		'''
		Add docstring
		'''
		# TODO : See the computationally efficient implementation

		self.get_dark_channel()

		# Calculate distribution 
		frequencies = np.zeros(256).astype(int)
		for i in range(0,self.dark_channel.shape[0]):
			for j in range(0,self.dark_channel.shape[1]):
				frequencies[int(self.dark_channel[i,j,0])]+=1

		assert sum(frequencies) == self.image.shape[0]*self.image.shape[1]
		
		# print('Frequencies : {}'.format(frequencies))

		# Find the top 1% brightest pixels in the dark channel image
		threshold = self.threshold_percentage*sum(frequencies)
		# print('Threshold : {}'.format(threshold))

		rev_freq = np.flip(frequencies,axis=0)
		cum_sum = np.cumsum(rev_freq)
		index = np.array(np.where(cum_sum <= threshold)).reshape(-1) # Stores the gray values which are in the top threshold percentage

		top_gray_values = []
		sum_so_far = 0

		for i in range(len(cum_sum)):
			if rev_freq[i] == 0:
				continue
			sum_so_far+=rev_freq[i]
			top_gray_values.append(255 - i)
			if sum_so_far > threshold:
				break

		# # Seperate gray values not present in the dark channel image
		# for i in index:
		# 	if rev_freq[i] == 0:
		# 		continue
		# 	top_gray_values.append(255 - i)
		
		# print('Top Gray Values : {}'.format(top_gray_values))
		A = np.zeros((1,3))

		max_in_b = max_in_g = max_in_r = -1.0*float('inf')

		# Find maximum for each channel across these values
		for tg in top_gray_values:
			indices = np.where(self.dark_channel == tg)
			iterater_ = zip(indices[0],indices[1])
	
			for i,j in iterater_:
				c = 0 # Channel under consideration
				if(self.image[i,j,c] > max_in_b):
					max_in_b = self.image[i,j,c]
				c = 1
				if(self.image[i,j,c] > max_in_g):
					max_in_g = self.image[i,j,c]
				c = 2
				if(self.image[i,j,c] > max_in_r):
					max_in_r = self.image[i,j,c]

		A[0,0] = max_in_b
		A[0,1] = max_in_g
		A[0,2] = max_in_r

		self.A = A.reshape(1,1,3)

		if self.show_intermediate is True:
			print(A)

	def get_transmission_map(self):
		self.get_A()
		normalized_haze_image = self.image/self.A
		w = self.w

		self.transmission_map = np.zeros((normalized_haze_image.shape[0],normalized_haze_image.shape[1],1))
		padded_image = np.pad(normalized_haze_image,((int(w/2),int(w/2)),(int(w/2),int(w/2)),(0,0)),'edge')

		for i in range(int(w/2),padded_image.shape[0] - int(w/2)):
			for j in range(int(w/2),padded_image.shape[1] - int(w/2)):
				block = padded_image[i-int(w/2):i+int(w/2)+1,j-int(w/2):j+int(w/2)+1,:]
				self.transmission_map[i-int(w/2),j-int(w/2),0] = 1.0 - self.omega*np.min(block)

		if self.show_intermediate is True:
			print(self.transmission_map)
			cv2.imshow('Transmission Map',self.transmission_map)
			cv2.waitKey(0)

	def mean_filter(self,I, w):
		kernel = np.ones((w,w),np.float32)/(1.0*w*w)
		return cv2.filter2D(I,-1,kernel)

	def guided_filter(self,I, p):
	    """Refine a filter under the guidance of another (RGB) image.
	    Parameters
	    -----------
	    I:   an M * N * 3 RGB image for guidance.
	    p:   the M * N filter to be guided
	    r:   the radius of the guidance
	    eps: epsilon for the guided filter
	    Return
	    -----------
	    The guided filter.
	    """
	    R,G,B = 0, 1, 2
	    w = self.w_refine
	    eps = self.epsilon

	    p = p.reshape(p.shape[0],p.shape[1])
	    M, N = self.image.shape[0],self.image.shape[1]
	    # base = self.mean_filter(np.ones((M, N)), w)

	    # print('Base : {}'.format(base))
	    # means = [self.mean_filter(I[:, :, i], w) / base for i in range(3)]
	    # print(means)

	    # each channel of I filtered with the mean filter
	    means = [self.mean_filter(I[:, :, i], w) for i in range(3)] # mu_k
	    # print(means)

	    # p filtered with the mean filter
	    mean_p = self.mean_filter(p, w) # p_k_bar
	    # filter I with p then filter it with the mean filter
	    means_IP = [self.mean_filter(I[:, :, i] * p, w) for i in range(3)] # First term in second bracket of equation 14
	    # covariance of (I, p) in each local patch
	    covIP = [means_IP[i] - means[i] * mean_p for i in range(3)] # Second term in equation 14

	    '''
	    Equation 14 describes an image as the second term...the summation is nothing but a mean filter applied to every pixel
	    '''

	    # variance of I in each local patch: the matrix Sigma in eq.14
	    var = defaultdict(dict)
	    for i, j in combinations_with_replacement(range(3), 2):
	        var[i][j] = self.mean_filter(
	            I[:, :, i] * I[:, :, j], w) - means[i] * means[j] # Matrix Sigma terms for each pixel

	    a = np.zeros((M, N, 3))
	    for y, x in np.ndindex(M, N):
	        #         rr, rg, rb
	        # Sigma = rg, gg, gb
	        #         rb, gb, bb
	        Sigma = np.array([[var[R][R][y, x], var[R][G][y, x], var[R][B][y, x]],
	                          [var[R][G][y, x], var[G][G][y, x], var[G][B][y, x]],
	                          [var[R][B][y, x], var[G][B][y, x], var[B][B][y, x]]])
	        cov = np.array([c[y, x] for c in covIP])
	        a[y, x] = np.dot(cov, inv(Sigma + eps * np.eye(3)))  # eq 14
	    
	    b = mean_p - a[:, :, R] * means[R] - a[:, :, G] * means[G] - a[:, :, B] * means[B] # Equation 15

	    q = (self.mean_filter(a[:, :, R], w) * I[:, :, R] + self.mean_filter(a[:, :, G], w) *
	         I[:, :, G] + self.mean_filter(a[:, :, B], w) * I[:, :, B] + self.mean_filter(b, w)) # Equations 7,8,16

	    self.refined_transmission_map = q.reshape(q.shape[0],q.shape[1],1)

	def refine_transmission_map(self):
		normalized_image = (self.image - self.image.min())/(self.image.max() - self.image.min())
		self.guided_filter(normalized_image,self.transmission_map)

		if self.show_intermediate is True:
			print(self.refined_transmission_map)
			cv2.imshow('Refined Transmission Map',self.refined_transmission_map)
			cv2.waitKey(0)

	def dehaze(self):
		self.get_transmission_map()

		if self.refine is True:
			self.refine_transmission_map()

		# Normalizing I and A
		self.image = (self.image - np.min(self.image))/(np.max(self.image) - np.min(self.image))
		self.A/=255.0

		if self.refine is True:
			self.J = (self.image - self.A)/(np.maximum(self.refined_transmission_map,self.t0)) + self.A
		else:
			self.J = (self.image - self.A)/(np.maximum(self.transmission_map,self.t0)) + self.A

		if self.show_intermediate is True:
			print(self.J)
			cv2.imshow('Dehazed Image',self.J)
			cv2.waitKey(0)

		return self.J*255.0

In [5]:
import cv2;
import math;
import numpy as np;

# 求出暗通道
def DarkChannel(im,sz):
    """
    im:输入图像
    sz:窗口大小
    """
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

# 求出全球大气光值A
def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz);
    imvec = im.reshape(imsz,3);

    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx;
    return A

def TransmissionEstimate(im,A,sz):
    omega = 0.95;
    im3 = np.empty(im.shape,im.dtype);

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz);
    return transmission

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);

    return t;

def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]

    return res

if __name__ == '__main__':
    fn = 'T:\GitHub\DIP_lv\Experiment2\img\haze.png'


    src = cv2.imread(fn);

    I = src.astype('float64')/255;
 
    dark = DarkChannel(I,15);
    A = AtmLight(I,dark);
    te = TransmissionEstimate(I,A,15);
    t = TransmissionRefine(src,te);
    J = Recover(I,t,A,0.1);

    cv2.imshow("dark",dark);
    cv2.imshow("t",t);
    cv2.imshow('I',src);
    cv2.imshow('J',J);
    cv2.imwrite("./image/J.png",J*255);
    cv2.waitKey();

In [ ]:
import cv2
import math
import numpy as np

#获取暗通道
def getDarkChannel(img,windowSize):
    """
    img:输入图像
    windowSize:窗口大小
    """
    b,g,r=cv2.split(img)
    darkC=cv2.min(cv2.min(r,g),b)
    """
    #cv2.getStructuringElement( ) 返回指定形状和尺寸的结构元素
    矩形:MORPH_RECT;
    交叉形:MORPH_CROSS;
    椭圆形:MORPH_ELLIPSE;
    第二和第三个参数分别是内核的尺寸以及锚点的位置
    """
    kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(windowSize,windowSize))
    """
    二值图像f(x,y) 的膨胀操作，类似于对图像的卷积操作.
    需要有个 kernel 操作矩阵，类似于卷积核(filters, kernel)，常见的是 3X3 的矩阵. 这是形态学处理的核心.
    但与卷积不同的是，如果矩阵中的像素点有任意一个点的值是前景色，则设置中心像素点为前景色；否则不变.
    最小值卷积相当于腐蚀操作
    """
    darkChannel=cv2.erode(darkC,kernel)
    return darkChannel

# 求出全球大气光值
def AtmLight(img,darkChannel):
    """
    img:图像
    darkChannel: 暗通道
    """
    [h,w]=img.shape[:2]
    imgSize=h*w
    numpx = int(max(math.floor(imgSize/1000),1))
    darkvec = dark.reshape(imgSize)
    imvec = img.reshape(imgSize,3)

    indices = darkvec.argsort()
    indices = indices[imgSize-numpx::]

    atmsum = np.zeros([1,3])
    for channel_index in range(1,numpx):
       atmsum = atmsum + imvec[indices[channel_index]]

    A = atmsum / numpx
    return A

def TransmissionEstimate(im,A,sz):
    omega = 0.95
    im3 = np.empty(im.shape,im.dtype)
    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*getDarkChannel(im3,sz)
    return transmission

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r))
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r))
    cov_Ip = mean_Ip - mean_I*mean_p

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r))
    var_I   = mean_II - mean_I*mean_I

    a = cov_Ip/(var_I + eps)
    b = mean_p - a*mean_I

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r))
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r))

    q = mean_a*im + mean_b
    return q;

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray)/255
    r = 60
    eps = 0.0001
    t = Guidedfilter(gray,et,r,eps)

    return t

def recover(img,t,A,tx=0.1):
    """
    img:雾图像
    t: 投射图
    A: 大气光值
    tx: 投射图阈值
    """
    result=np.empty(img.shape,img.dtype)
    t=cv2.max(t,tx)

    for channel_index in range(0,3):
        result[:,:,channel_index]=(img[:,:,channel_index]-A[0,channel_index])/t+A[0:channel_index]
    
    return result

if __name__ == '__main__':
    fn = 'T:\GitHub\DIP_lv\Experiment2\img\haze.png'


    src = cv2.imread(fn);

    I = src.astype('float64')/255;
 
    dark = getDarkChannel(I,15);
    A = AtmLight(I,dark);
    te = TransmissionEstimate(I,A,15);
    t = TransmissionRefine(src,te);
    J = recover(I,t,A,0.1);

    cv2.imshow("dark",dark);
    cv2.imshow("t",t);
    cv2.imshow('I',src);
    cv2.imshow('J',J);
    cv2.imwrite("T:\GitHub\DIP_lv\Experiment2\img\J.png",J*255);
    cv2.waitKey();